In [1]:
import numpy as np
from uncertainties import ufloat
from uncertainties import umath
from uncertainties import wrap
from numpy import sqrt 
from numpy import pi
from scipy.integrate import trapz
import emcee
import corner
import matplotlib.pyplot as plt
from lmfit import Model
import lmfit
from lmfit import Parameters
import pandas as pd
from scipy.signal import savgol_filter
from scipy import signal
from scipy.stats import sem
import math
import time
import itertools

In [2]:
speed_of_light=2.99792e5

rest_wl_si_1=5957.56
rest_wl_si_2=5978.93

def vel_to_wl(vel,rest_wl_line):
    zmeas=-1.0+np.sqrt( (1.0+(vel/speed_of_light))/(1.0-(vel/speed_of_light)) )
    wl=(zmeas*rest_wl_line)+rest_wl_line
    return wl
def wl_to_vel(wl,rest_wl_line):
    zmeas=(wl-rest_wl_line)/rest_wl_line
    vel=(((zmeas+1.0)**2-1.0)*speed_of_light/(1.+(1.+zmeas)**2))
    return vel
def g_sil1(x,a_sil1,v_sil1,fwhm_sil1):
    zmeas=(x-rest_wl_si_1)/rest_wl_si_1
    vel=(( (zmeas+1.0)**2 - 1.0 )*speed_of_light/(1.+(1.+zmeas)**2))/1e4
    sigma_gaussian=np.abs(fwhm_sil1)/2.35482
    gaussian=a_sil1*np.exp(-np.power(vel - v_sil1, 2.) / (2 * np.power(sigma_gaussian, 2.)))
    return gaussian
def g_sil2(x,a_sil2,v_sil2,fwhm_sil2):
    zmeas=(x-rest_wl_si_2)/rest_wl_si_2
    vel=(( (zmeas+1.0)**2 - 1.0 )*speed_of_light/(1.+(1.+zmeas)**2))/1e4
    sigma_gaussian=np.abs(fwhm_sil2)/2.35482
    gaussian=a_sil2*np.exp(-np.power(vel - v_sil2, 2.) / (2 * np.power(sigma_gaussian, 2.)))
    return gaussian

In [3]:
mean_sil_vel_ph = []
mean_sil_vel_ph_err = []

median_sil_vel_ph = []
median_sil_vel_ph_err = []

sil_vel_lower_err = []
sil_vel_higher_err = []

mean_sil_fwhm_ph = []
mean_sil_fwhm_ph_err = []

median_sil_fwhm_ph = []
median_sil_fwhm_ph_err = []

mean_ph_pEW = []
mean_ph_pEW_err = []

median_ph_pEW = []
median_ph_pEW_err = []

median_pEW_trap = []
median_pEW_trap_err = []

pEW_lower_err = []
pEW_higher_err = []

pEW_trap_lower_err = []
pEW_trap_higher_err = []

fit_sil_vel_err_std = []
fit_pEW_err_std = []
fit_FWHMA_err_std = []

mean_sil_fwhmA_ph = []
mean_sil_fwhmA_ph_err = []

median_sil_fwhmA_ph = []
median_sil_fwhmA_ph_err = []

fwhmA_lower_err = []
fwhmA_higher_err =[]

mean_x_1 = []
mean_x_2 = []

    
SD_vel = []

SD_pEW = []

SD_FWHMA = []

# spec_log='/Users/umutburgaz/ZTFWork/Ia_sample_final/5972/oneone.txt'
#spec_log='/Users/umutburgaz/ZTFWork/Ia_sample_final/5972/Current_May_29/just_one.txt'
# df = pd.read_csv(spec_log,delim_whitespace=True)

#df = pd.read_csv("/Users/umutburgaz/ZTFWork/25_Jun/fit_one.csv") umut original 
#df = pd.read_csv(r"C:\Users\senan\OneDrive\Desktop\Capstone\dereddened_files_est_5972.csv")
df = pd.read_csv(r"dereddened_files_est_5972_111-112.csv")

filename = df['filename']
ztfname = df['ztfname']
z = df['redshift']
phase = df['phase']

vx1 = df['v_x1_5972']
vx2 = df['v_x2_5972']

#%%
st = time.time()

ahh = 'hello\goodbye'  # just to get the slash in a string
slash = ahh[5] 
path_stem = r'C:\Users\senan\OneDrive\Desktop\Capstone\dereddend_spectra_and_Si_figures'

for idx in range(0,len(filename),1):
#for idx in range(36,37,1):
    iteration = idx
    print("iteration", iteration)
#for idx in range(0,2,1):
    #spec_file = '/Users/umutburgaz/ZTFWork/a_May_28_new/mw_all/{}'.format(filename[idx]) umut original 
    
    #data = np.loadtxt(spec_file,dtype={'names': ('wl', 'fl'),'formats': ('f8', 'f8')} ) umut_origianl
    #wl_fit1=data['wl']/(1+ z[idx])#z loop this                                             umut_original
    #fl_fit1=data['fl']                                 	                                umut_original
    #flerr_fit1=data['fl']*0.01                                                             umut_original
    
    spec_file = path_stem + slash + filename[idx]

    data = pd.read_csv(spec_file)
    wl_fit1=( np.array(data['wl']) ) / (1+z[idx])
    fl_fit1=np.array(data['fl'])
    flerr_fit1=( np.array(data['fl']) ) * 0.01
    
    print(filename[idx])
    print(z[idx])

    vx1 = df['v_x1_5972'][idx]

#   vx1_arr = np.random.uniform(low=vx1-600, high=vx1+600, size=32)
    vx1_arr = np.random.normal(loc=vx1, scale=400, size=32)
    
    vx2 = df['v_x2_5972'][idx]

#   vx2_arr = np.random.uniform(low=vx2-600, high=vx2+600, size=32)
    vx2_arr = np.random.normal(loc=vx2, scale=400, size=32)
    
    
    sil_vel_ph = []
    sil_vel_ph_err = []

    sil_fwhm_ph = []
    sil_fwhm_ph_err = []

    new_ph_pEW = []
    new_ph_pEW_err = []
    
    new_ph_fwhmA = []
    new_ph_fwhmA_err = []
    
    pEW_trap = []
    pEW_trap_err = []
    
    x_1 = []
    x_2 = []
    
    lower_list_pEW = []
    higher_list_pEW = []
    
    lower_list_pEW_trap = []
    higher_list_pEW_trap = []
    
    lower_list_sil = []
    higher_list_sil = []
    
    lower_list_fwhmA = []
    higher_list_fwhmA = []

    for ind1 in range(len(vx1_arr)):
        for ind2 in range(len(vx2_arr)):
            
    
            v_x1= vx1_arr[ind1]          
            
            v_x2= vx2_arr[ind2]     
            
            if "_SEDm_" in spec_file:
                width_vel = 4000
            elif "_LT_" in spec_file or "_Other_" in spec_file:
                width_vel = 2000          
            else:
                width_vel = 1000
            
            line_back1=vel_to_wl(v_x1,rest_wl_si_1)
            line_back2=vel_to_wl(v_x2,rest_wl_si_1)
            
            line_back1a=vel_to_wl(v_x1-width_vel/2.0,rest_wl_si_1)
            line_back1b=vel_to_wl(v_x1+width_vel/2.0,rest_wl_si_1)
            
            line_back2a=vel_to_wl(v_x2-width_vel/2.0,rest_wl_si_1)
            line_back2b=vel_to_wl(v_x2+width_vel/2.0,rest_wl_si_1)
            
            wcont1= (( (wl_fit1 >= line_back1a) & (wl_fit1 <= line_back1b)) | ( (wl_fit1 >= line_back2a) & (wl_fit1 <= line_back2b)))
            
            xcont1=wl_fit1[wcont1]
            ycont1=fl_fit1[wcont1]
            yerrcont=flerr_fit1[wcont1]
            cont_res = np.polyfit(xcont1,ycont1,1,w=1.0/yerrcont)
            
            w=np.where( np.logical_and(wl_fit1 > line_back1,wl_fit1 < line_back2))
            x=wl_fit1[w]
            
            cont_fit=cont_res[1]+x*cont_res[0]
            cont2=cont_res[1]+wl_fit1*cont_res[0]
            y=(fl_fit1[w]/cont_fit)-1
            yerr=flerr_fit1[w]/cont_fit
            
            i=[-0.4,-1.0,1.0]
            
            try:
                model = Model(g_sil1)+Model(g_sil2)
    
                pars = Parameters()
                pars.add('a_sil1', value=i[0], max=0.0)
                pars.add('v_sil1', value=i[1], min=-4.0, max=0.0)
                pars.add('fwhm_sil1', value=i[2], min=0.0)
                pars.add('a_sil2', expr='a_sil1')
                pars.add('v_sil2', expr='v_sil1')
                pars.add('fwhm_sil2', expr='fwhm_sil1')
                out = model.fit(y, pars, x=x, weights=1.0/yerr)
    
                s1=(g_sil1(x,out.params['a_sil1'].value,out.params['v_sil1'].value,out.params['fwhm_sil1'].value))
                s2=(g_sil2(x,out.params['a_sil2'].value,out.params['v_sil2'].value,out.params['fwhm_sil2'].value))         
                    
                
                pEW1 = (trapz(abs(s1), x=x) + trapz(abs(s2), x=x))

                num_iterations = 100  # Number of Monte Carlo iterations

                # Create empty arrays to store the pEW1 values from each iteration
                pEW1_values = np.zeros(num_iterations)

                for i in range(num_iterations):
                    # Perturb the parameters using their uncertainties
                    perturbed_v_sil1 = np.random.normal(out.params['v_sil1'].value, out.params['v_sil1'].stderr)
                    perturbed_fwhm_sil1 = np.random.normal(out.params['fwhm_sil1'].value, out.params['fwhm_sil1'].stderr)
                    perturbed_a_sil1 = np.random.normal(out.params['a_sil1'].value, out.params['a_sil1'].stderr)

                    # Calculate the perturbed Gaussian profiles
                    s1_perturbed = g_sil1(x, perturbed_a_sil1, perturbed_v_sil1, perturbed_fwhm_sil1)
                    s2_perturbed = g_sil2(x, perturbed_a_sil1, perturbed_v_sil1, perturbed_fwhm_sil1)

                    # Calculate the perturbed pEW1
                    pEW1_perturbed = trapz(abs(s1_perturbed), x=x) + trapz(abs(s2_perturbed), x=x)
                    pEW1_values[i] = pEW1_perturbed

                # Calculate the mean and standard deviation of pEW1 values from the Monte Carlo iterations
                pEW1_mean = np.mean(pEW1_values)
                pEW1_stddev = np.std(pEW1_values)

                pEW_trap.append(pEW1)
                pEW_trap_err.append(pEW1_stddev)
                
                report= out.fit_report()
            
                # pEW calculation from the data
                sil_min_x1 = vel_to_wl((out.params['v_sil1'].value*1e4)-(out.params['fwhm_sil1'].value*1e4)/2, rest_wl_si_1)
                wrapped_f1 = wrap(vel_to_wl)
                v=ufloat(out.params['v_sil1'].value*1e4,out.params['v_sil1'].stderr*1e4)
                foo=wrapped_f1(v,rest_wl_si_1)
                sil_min_x1_err = foo.s
                
                sil_min_x2 = vel_to_wl((out.params['v_sil1'].value*1e4)+(out.params['fwhm_sil1'].value*1e4)/2, rest_wl_si_1)
                wrapped_f2 = wrap(vel_to_wl)
                v1=ufloat(out.params['v_sil1'].value*1e4,out.params['v_sil1'].stderr*1e4)
                foo1=wrapped_f2(v1,rest_wl_si_1)
                sil_min_x2_err =foo1.s
                
                amp = ufloat(out.params['a_sil1'].value, out.params['a_sil1'].stderr)
                sil_min_x1_c = ufloat(sil_min_x1, sil_min_x1_err)
                sil_min_x2_c = ufloat(sil_min_x2, sil_min_x2_err)
                
                fwhmA = sil_min_x2_c - sil_min_x1_c
                sigmaA = fwhmA / 2.35482
                pEW = 2*abs(amp)*abs(sigmaA)/0.3989
                             
                sil_vel_ph.append(out.params['v_sil1'].value)
                sil_vel_ph_err.append(out.params['v_sil1'].stderr)
                     
                sil_fwhm_ph.append(out.params['fwhm_sil1'].value)
                sil_fwhm_ph_err.append(out.params['fwhm_sil1'].stderr)
                
                new_ph_pEW.append(pEW.nominal_value)
                new_ph_pEW_err.append(pEW.s)
                
                new_ph_fwhmA.append(fwhmA.nominal_value)
                new_ph_fwhmA_err.append(fwhmA.s)
            except:
                sil_vel_ph.append(0.0)
                sil_vel_ph_err.append(0.0)
                     
                sil_fwhm_ph.append(0.0)
                sil_fwhm_ph_err.append(0.0)
                
                new_ph_pEW.append(0.0)
                new_ph_pEW_err.append(0.0)
                
                new_ph_fwhmA.append(0.0)
                new_ph_fwhmA_err.append(0.0)
                
                pEW_trap.append(0.0)
                pEW_trap_err.append(0.0)
                
    iteration = idx + 1
    
    try:
        plt.hist(sil_vel_ph, bins=10)
        plt.axvline(x=sil_vel_ph[0],c='red', label='first')
        plt.axvline(x=np.mean(sil_vel_ph), c='green', label='mean')
        plt.axvline(x=np.median(sil_vel_ph), c='purple', label='median')
        plt.legend()
        plt.savefig(spec_file+'_5972_hist_vel.png', bbox_inches='tight')
        plt.close()
    
        plt.hist(new_ph_pEW, bins=10)
        plt.axvline(x=new_ph_pEW[0],c='red', label='first')
        plt.axvline(x=np.mean(new_ph_pEW), c='green', label='mean')
        plt.axvline(x=np.median(new_ph_pEW), c='purple', label='median')
        plt.axvline(x=np.median(pEW_trap), c='blue', label='trap')

        plt.legend()
        plt.savefig(spec_file+'_5972_hist_pEW.png', bbox_inches='tight')
        plt.close()
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1) 
        plt.plot(wl_fit1,fl_fit1, 'o')
        mask1 = np.where((wl_fit1 >= 5200) & (wl_fit1 <= 7000))
        y_range1 = fl_fit1[mask1]
        plt.plot(xcont1,ycont1,'o', color='red')
        plt.plot(x,(y+1.0)*cont_fit)
        plt.plot(x,(s1+1.0)*cont_fit,':')
        plt.plot(x,(s2+1.0)*cont_fit,':')
        plt.plot(x,(s1+s2+1.0)*cont_fit)
        plt.xlim(line_back1-1000,line_back2+1000)
        plt.ylim(y_range1.min() - y_range1.min()*0.1, y_range1.max() + y_range1.max()*0.1)
        plt.subplot(1, 2, 2)
        plt.plot(wl_fit1,(fl_fit1/cont2)-1.0,'b-')
        mask2 = np.where((wl_fit1 >= 5500) & (wl_fit1 <= 6500))
        y_range2 = ((fl_fit1/cont2)-1.0)[mask2]            
        plt.plot([line_back1,line_back2],[0,0],'--')
        plt.plot(x,s1,':')
        plt.plot(x,s2,':')
        plt.plot(x,s1+s2, '-', color='black')
        plt.plot([line_back1,line_back1],[-10,10])
        plt.plot([line_back1a,line_back1a],[-10,10],":")
        plt.plot([line_back1b,line_back1b],[-10,10],":")
        plt.plot([line_back2,line_back2],[-10,10])
        plt.plot([line_back2a,line_back2a],[-10,10],":")
        plt.plot([line_back2b,line_back2b],[-10,10],":")
        #plt.ylim(-2,2)
        plt.ylim(y_range2.min() + y_range2.min()*0.1, y_range2.max() + y_range2.max()*0.1)
        plt.xlim(line_back1-200,line_back2+200)
        plt.tight_layout()
        plt.savefig(spec_file+'_5972_new.png', bbox_inches='tight')
        plt.close()
           
        mean_sil_vel_ph.append(np.mean(sil_vel_ph))
        sil_vel_ph_err = [w for w in sil_vel_ph_err if w is not None]
        mean_sil_vel_ph_err.append(np.nanmean(sil_vel_ph_err[sil_vel_ph_err != None]))
        
        median_sil_vel_ph.append(np.median(sil_vel_ph))
        sil_vel_ph_err = [w for w in sil_vel_ph_err if w is not None]
        median_sil_vel_ph_err.append(np.nanmedian(sil_vel_ph_err[sil_vel_ph_err != None]))
        
        from_fit_sil_vel_ph_err_std = np.std(sil_vel_ph_err)
        fit_sil_vel_err_std.append(from_fit_sil_vel_ph_err_std)
        
        SD_sil_vel = np.std(sil_vel_ph)
        SD_vel.append(SD_sil_vel)
    
        mean_sil_fwhm_ph.append(np.nanmean(sil_fwhm_ph))
        sil_fwhm_ph_err = [w for w in sil_fwhm_ph_err if w is not None]
        mean_sil_fwhm_ph_err.append(np.nanmean(sil_fwhm_ph_err[sil_fwhm_ph_err != None]))
        
        median_sil_fwhm_ph.append(np.nanmedian(sil_fwhm_ph))
        sil_fwhm_ph_err = [w for w in sil_fwhm_ph_err if w is not None]
        median_sil_fwhm_ph_err.append(np.nanmedian(sil_fwhm_ph_err[sil_fwhm_ph_err != None]))
    
        mean_ph_pEW.append(np.nanmean(new_ph_pEW))
        new_ph_pEW_err = [w for w in new_ph_pEW_err if w is not None]
        mean_ph_pEW_err.append(np.nanmean(new_ph_pEW_err[new_ph_pEW_err != None]))
        
        median_ph_pEW.append(np.nanmedian(new_ph_pEW))
        new_ph_pEW_err = [w for w in new_ph_pEW_err if w is not None]
        median_ph_pEW_err.append(np.nanmedian(new_ph_pEW_err[new_ph_pEW_err != None]))
        
        median_pEW_trap.append(np.nanmedian(pEW_trap))
        pEW_trap_err = [w for w in pEW_trap_err if w is not None]
        median_pEW_trap_err.append(np.nanmedian(pEW_trap_err[pEW_trap_err != None]))
        
        from_fit_pEW_err_std = np.std(new_ph_pEW_err)
        fit_pEW_err_std.append(from_fit_pEW_err_std)
        
        SD_pEW_samp = np.std(new_ph_pEW)
        SD_pEW.append(SD_pEW_samp)
    
        mean_sil_fwhmA_ph.append(np.nanmean(new_ph_fwhmA))
        new_ph_fwhmA_err = [w for w in new_ph_fwhmA_err if w is not None]
        mean_sil_fwhmA_ph_err.append(np.nanmean(new_ph_fwhmA_err[new_ph_fwhmA_err != None]))
        
        median_sil_fwhmA_ph.append(np.nanmedian(new_ph_fwhmA))
        new_ph_fwhmA_err = [w for w in new_ph_fwhmA_err if w is not None]
        median_sil_fwhmA_ph_err.append(np.nanmedian(new_ph_fwhmA_err[new_ph_fwhmA_err != None]))

        from_fit_FWHMA_err_std = np.std(new_ph_fwhmA_err)
        fit_FWHMA_err_std.append(from_fit_FWHMA_err_std)
        
        SD_FWHMA_samp = np.std(new_ph_fwhmA)
        SD_FWHMA.append(SD_FWHMA_samp)
        
        mean_x_1.append(np.nanmean(x_1))
        mean_x_2.append(np.nanmean(x_2))
        
        median_value_pEW = np.nanmedian(pEW_trap)
        lower_list_pEW = [value for value in new_ph_pEW if value < median_value_pEW]
        higher_list_pEW = [value for value in new_ph_pEW if value >= median_value_pEW]
        lower_list_pEW_sd = np.std(lower_list_pEW)
        higher_list_pEW_sd = np.std(higher_list_pEW)
        
        pEW_lower_err.append(lower_list_pEW_sd)
        pEW_higher_err.append(higher_list_pEW_sd)
        
        median_value_pEW_trap = np.nanmedian(pEW_trap)
        lower_list_pEW_trap = [value for value in pEW_trap if value < median_value_pEW_trap]
        higher_list_pEW_trap = [value for value in pEW_trap if value >= median_value_pEW_trap]
        lower_list_pEW_trap_sd = np.std(lower_list_pEW_trap)
        higher_list_pEW_trap_sd = np.std(higher_list_pEW_trap)
        
        pEW_trap_lower_err.append(lower_list_pEW_trap_sd)
        pEW_trap_higher_err.append(higher_list_pEW_trap_sd)

        median_value_sil = np.nanmedian(sil_vel_ph)
        lower_list_sil = [value for value in sil_vel_ph if value < median_value_sil]
        higher_list_sil = [value for value in sil_vel_ph if value >= median_value_sil]
        lower_list_sil_sd = np.std(lower_list_sil)
        higher_list_sil_sd = np.std(higher_list_sil)
        
        sil_vel_lower_err.append(lower_list_sil_sd)
        sil_vel_higher_err.append(higher_list_sil_sd)
        
        median_value_fwhmA = np.nanmedian(new_ph_fwhmA)
        lower_list_fwhmA = [value for value in new_ph_fwhmA if value < median_value_fwhmA]
        higher_list_fwhmA = [value for value in new_ph_fwhmA if value >= median_value_fwhmA]
        lower_list_fwhmA_sd = np.std(lower_list_fwhmA)
        higher_list_fwhmA_sd = np.std(higher_list_fwhmA)
        
        fwhmA_lower_err.append(lower_list_fwhmA_sd)
        fwhmA_higher_err.append(higher_list_fwhmA_sd)        
        
        
    except:
        mean_sil_vel_ph.append(0.0)
        sil_vel_ph_err = [w for w in sil_vel_ph_err if w is not None]
        mean_sil_vel_ph_err.append(0.0)
        
        median_sil_vel_ph.append(0.0)
        sil_vel_ph_err = [w for w in sil_vel_ph_err if w is not None]
        median_sil_vel_ph_err.append(0.0)
        
        SD_vel.append(0.0)
    
        mean_sil_fwhm_ph.append(0.0)
        sil_fwhm_ph_err = [w for w in sil_fwhm_ph_err if w is not None]
        mean_sil_fwhm_ph_err.append(None)
        
        median_sil_fwhm_ph.append(0.0)
        sil_fwhm_ph_err = [w for w in sil_fwhm_ph_err if w is not None]
        median_sil_fwhm_ph_err.append(0.0)
    
        mean_ph_pEW.append(0.0)
        new_ph_pEW_err = [w for w in new_ph_pEW_err if w is not None]
        mean_ph_pEW_err.append(0.0)
        
        median_ph_pEW.append(0.0)
        new_ph_pEW_err = [w for w in new_ph_pEW_err if w is not None]
        median_ph_pEW_err.append(0.0)
        
        SD_pEW.append(0.0)
    
        mean_sil_fwhmA_ph.append(0.0)
        new_ph_fwhmA_err = [w for w in new_ph_fwhmA_err if w is not None]
        mean_sil_fwhmA_ph_err.append(0.0)
        
        median_sil_fwhmA_ph.append(0.0)
        new_ph_fwhmA_err = [w for w in new_ph_fwhmA_err if w is not None]
        median_sil_fwhmA_ph_err.append(0.0)
        median_pEW_trap.append(0.0)
        median_pEW_trap_err.append(0.0)
        mean_x_1.append(0.0)
        mean_x_2.append(0.0)
        
        pEW_trap_lower_err.append(0.0)
        pEW_trap_higher_err.append(0.0)
        pEW_lower_err.append(0.0)
        pEW_higher_err.append(0.0)

        sil_vel_lower_err.append(0.0)
        sil_vel_higher_err.append(0.0)
        
        fwhmA_lower_err.append(0.0)
        fwhmA_higher_err.append(0.0)        

et = time.time()
elapsed_time = et - st #23 sec for 3000 , 15.6 sec for 2000 , 8 sec for 1000
print('Execution time:', elapsed_time, 'seconds')
print('Execution time in mins:', elapsed_time/60, 'minutes')
#%%

#CREATE DF HERE

final_df = pd.DataFrame(
    {'ztfname': df['ztfname'],
     'filename': df['filename'],
     'redshift': df['redshift'],
     'phase':df['phase'],
     'v_x1': df['v_x1_5972'],
     'v_x2': df['v_x2_5972'],
     
     'V_Sil_5972_mean':  mean_sil_vel_ph,
     'V_err_Sil_5972_mean': mean_sil_vel_ph_err,
     
     'V_Sil_5972_median':  median_sil_vel_ph,
     'V_err_Sil_5972_median': median_sil_vel_ph_err,
     'V_lower_err_Sil_5972_median': sil_vel_lower_err,
     'V_higher_err_Sil_5972_median': sil_vel_higher_err,

     'pEW_Sil_5972_mean': mean_ph_pEW,
     'pEW_err_Sil_5972_mean': mean_ph_pEW_err,
     
     'pEW_Sil_5972_median': median_ph_pEW,
     'pEW_err_Sil_5972_median': median_ph_pEW_err,
     'pEW_lower_err_Sil_5972_median': pEW_lower_err,
     'pEW_higher_err_Sil_5972_median': pEW_higher_err,
     'pEW_Sil_5972_median_trap': median_pEW_trap,
     'pEW_Sil_5972_median_trap_err': median_pEW_trap_err,
     'pEW_lower_trap_err_Sil_5972_median': pEW_trap_lower_err,
     'pEW_higher_trap_err_Sil_5972_median': pEW_trap_higher_err,    
     
     'FWHMv_Sil_5972_mean': mean_sil_fwhm_ph,
     'FWHMv_err_Sil_5972_mean': mean_sil_fwhm_ph_err,
     
     'FWHMv_Sil_5972_median': median_sil_fwhm_ph,
     'FWHMv_err_Sil_5972_median': median_sil_fwhm_ph_err,
     
     'FWHMA_Sil_5972_mean': mean_sil_fwhmA_ph,
     'FWHMA_err_Sil_5972_mean': mean_sil_fwhmA_ph_err,
     
     'FWHMA_Sil_5972_median': median_sil_fwhmA_ph,
     'FWHMA_err_Sil_5972_median': median_sil_fwhmA_ph_err,
     'FWHMA_lower_err_Sil_5972_median': fwhmA_lower_err,
     'FWHMA_higher_err_Sil_5972_median': fwhmA_higher_err,
    })

#creates file
output_csv = final_df
output_csv

new_pEW_error_values5 = np.sqrt((final_df['pEW_Sil_5972_median_trap_err']**2) + (final_df['pEW_lower_trap_err_Sil_5972_median']**2))
new_pEW_error_values6 = np.sqrt((final_df['pEW_Sil_5972_median_trap_err']**2) + (final_df['pEW_higher_trap_err_Sil_5972_median']**2))

new_FWHMA_error_values5 = np.sqrt((final_df['FWHMA_err_Sil_5972_median']**2) + (final_df['FWHMA_lower_err_Sil_5972_median']**2))
new_FWHMA_error_values6 = np.sqrt((final_df['FWHMA_err_Sil_5972_median']**2) + (final_df['FWHMA_higher_err_Sil_5972_median']**2))


new_vel_value11 = np.sqrt((final_df['V_err_Sil_5972_median']**2) + (final_df['V_lower_err_Sil_5972_median']**2))
new_vel_value1 = new_vel_value11 + 0.02

new_vel_value22 = np.sqrt((final_df['V_err_Sil_5972_median']**2) + (final_df['V_higher_err_Sil_5972_median']**2))
new_vel_value2 = new_vel_value22 + 0.02


final_df['V_Sil_err_low_5972_final'] = new_vel_value1
final_df['V_Sil_err_high_5972_final'] = new_vel_value2

final_df['pEW_err_trap_low_5972_final'] = new_pEW_error_values5
final_df['pEW_err_trap_high_5972_final'] = new_pEW_error_values6

final_df['FWHMA_err_low_5972_final'] = new_FWHMA_error_values5
final_df['FWHMA_err_high_5972_final'] = new_FWHMA_error_values6


#%%

#creates file
output_csv = final_df
output_csv

#final_df.to_csv("/Users/umutburgaz/ZTFWork/25_Jun/just1.csv", index=False) umut original
final_df.to_csv("5972_pEW_CalculatedValues111-112.csv") #already ran and file saved

#%%

iteration 0
ZTF20acquetr_20201116_P60_v1_dereddened.csv
0.031099


c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\senan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: RankWarning: 

TypeError: expected non-empty vector for x

## simplified version to use

In [4]:
#final_df = pd.DataFrame(
#    {'ztfname': df['ztfname'],
#     'filename': df['filename'],
#     'redshift': df['redshift'],
#     'phase':df['phase'],
#     'v_x1_5972': df['v_x1_5972'],
#     'v_x2_5972': df['v_x2_5972'],
#     'V_Sil_5972_median':  median_sil_vel_ph,
#     'pEW_Sil_5972_median_trap': median_pEW_trap,
#     'FWHMA_Sil_5972_median': median_sil_fwhmA_ph,
#     'V_lower_err_Sil_5972_median': sil_vel_lower_err,
#     'V_higher_err_Sil_5972_median': sil_vel_higher_err,
#     'pEW_lower_trap_err_Sil_5972_median': pEW_trap_lower_err,
#     'pEW_higher_trap_err_Sil_5972_median': pEW_trap_higher_err,    
#     'FWHMA_lower_err_Sil_5972_median': fwhmA_lower_err,
#     'FWHMA_higher_err_Sil_5972_median': fwhmA_higher_err,})
##%%
#
##creates file
#output_csv = final_df
#output_csv
#
#final_df.to_csv("/Users/umutburgaz/ZTFWork/25_Jun/just1_simp.csv", index=False)
#
##%%